# Movie Recommender

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

## Import the data

In [12]:
ratingsdf = pd.read_csv("Data/ratings.csv")
linksdf = pd.read_csv("Data/links.csv")
tagsdf = pd.read_csv("Data/tags.csv")
moviesdf = pd.read_csv("Data/movies.csv")

In [22]:
# WRONG moviesdf.join(linksdf, on="movieId", how="inner", lsuffix="", rsuffix="_link")

In [15]:
# userid, rating, 

### Find one user or aggregate many users  
    1) find the most similar user or aggregate user
    2) identify that user's highly rated movies

### Metrics to judge similarity
        - based on aggregrate rating of a movie
        - based on what the rating
        - based on what genres they watched the most of
        - based on how frequently they rate movies
        - something with tags on the movie? Sentiment analysis?
        - scrape data from imdb for a critic's review
        - timestamps?

### Misc  
    - User liked a movie = T/F if their rating > aggregate movie rating
    - 

In [10]:
def rec_func(data):
    pass